In [ ]:
import os
import numpy as np
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
from img_helper import reduce_to_1
import pickle

K.set_image_data_format('channels_last')
# Fix random seed for reproducibility
seed = 7
np.random.seed(seed)

dataset_npy_path = "Dataset npys/"

trained_models_path = "trained models/"
trained_models_history_path = "trained models history/"

test_ready_models_path = "test ready models/"
test_ready_models_history_path = "test ready models history/"

In [ ]:
# Image parameters
img_width = 100
img_height = img_width
img_channels = 3

In [ ]:
train_HSV_Y = np.load(os.path.join(dataset_npy_path,"train_HSV_Y.npy"))
valid_HSV_Y = np.load(os.path.join(dataset_npy_path,"valid_HSV_Y.npy"))

train_noisy_HSV_Y = np.load(os.path.join(dataset_npy_path,"train_noisy_HSV_Y.npy"))
valid_noisy_HSV_Y = np.load(os.path.join(dataset_npy_path,"valid_noisy_HSV_Y.npy"))
print("data loaded")

In [ ]:
# Convolutional encoder parameters
conv_epochs = 20
conv_batch_size = 128
conv_input_img = Input(shape = (img_width, img_height, 1))

# Convolutional encoder architecture
def convolutional_autoencoder(conv_input_img):
	conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_input_img)
	pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
	conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
	up1 = UpSampling2D((2,2))(conv2)
	conv3 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up1)
	return conv3

# Build convolutional model
conv_encoder = Model(conv_input_img, convolutional_autoencoder(conv_input_img))
conv_encoder.compile(loss='mean_squared_error', optimizer='Adam')

In [ ]:
# Train convolutional model
conv_encoder_h = conv_encoder.fit(reduce_to_1(train_noisy_HSV_Y, 2), reduce_to_1(train_HSV_Y, 2),
					batch_size=conv_batch_size,
					epochs=conv_epochs,
					verbose=1,
					validation_data=(reduce_to_1(valid_noisy_HSV_Y, 2), reduce_to_1(valid_HSV_Y, 2)))

In [ ]:
conv_encoder.save(os.path.join(trained_models_path,"conv_encoder.h5"))
print("Saved conv_encoder model to disk")

with open(os.path.join(trained_models_history_path,'trainHistoryDict_conv.pkl'), 'wb') as file_pi:
        pickle.dump(conv_encoder_h.history, file_pi)

In [ ]:
import shutil
test_ready_save_enable = False
if test_ready_save_enable:
    shutil.copy2(os.path.join(trained_models_path,"conv_encoder.h5"), os.path.join(test_ready_models,"conv_encoder_test.h5"))
    print("Saved conv_encoder model test copy to disk")

    shutil.copy2(os.path.join(trained_models_history_path,'trainHistoryDict_conv.pkl'), 
                 os.path.join(test_ready_models_history_path,'trainHistoryDict_conv_test.pkl'))